# 파이토치 실습

## 1. torchvision으로 데이터 불러오기

### 1.1 CNN 라이브러리 호출

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader


### 1.2 CPU 혹은 GPU 확인

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### 1.3 fashion_mnist 데이터셋 내려받기

In [ ]:
train_dataset = torchvision.datasets.FashionMNIST("./data",download=True,train=True,transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = torchvision.datasets.FashionMNIST("./data",download=True,train=False,transform=transforms.Compose([transforms.ToTensor()]))

### 1.4 fashion_mnist 데이터를 데이터로더에 전달

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=100)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=100)

### 1.5 분류에 사용될 클래스 정의

In [ ]:
labels_map = { 0 : 'T-Shirts', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress' , 4 : 'Coat', 5 : 'Sandal', 6 : 'Shirt', 7 : 'Sneaker', 8 : 'Bag' , 9 : 'Ankle Boot' }
fig = plt.figure(figsize=(8,8))
rows = 4
columns = 5 # 기둥
for i in range(1, rows*columns +1):
  img_xy = np.random.randint(len(train_dataset))
  img = train_dataset[img_xy][0][0,:,:]
  fig.add_subplot(rows, columns, i)
  plt.title(labels_map[train_dataset[img_xy][1]])
  # print("labels_map[train_dataset[img_xy][1]] : ",labels_map[train_dataset[img_xy][1]])
  # print("train_dataset[img_xy][1]:",train_dataset[img_xy][1])
  plt.axis('off')
  plt.imshow(img, cmap='gray')
plt.show()


* 파이썬 문법 - numpy#1

numpy.random.randint(low,max,sizeForm)

In [ ]:
# t = np.random.randint(1,7,(2,3))
# print(t)

### 1.6 직접실습

### 라이브러리 종류

torchvision.dataset에 내장된 라이브러리
1. Fashion_MNIST
2. CIFAR-10
3. ETC


In [ ]:
import numpy
import matplotlib.pyplot
import torch
import torchvision
import torchvision.transforms
import torch.utils

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
train_dset = torchvision.datasets.FashionMNIST("./practice", train=True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
test_dset = torchvision.datasets.FashionMNIST("./practice", train=False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))

60000


In [ ]:
print(len(train_dset))
print(len(test_dset))

60000
10000


In [ ]:
train_dset_loader = torch.utils.data.DataLoader(train_dset,batch_size=100) 
test_dset_loader = torch.utils.data.DataLoader(test_dset,batch_size=100)

In [ ]:
print(len(train_dset_loader))
print(len(test_dset_loader))

In [ ]:
import matplotlib.pyplot as plt

# 첫 번째 이미지를 가져옵니다.
for num in range(0,20):
  image, label = train_dset[num]

  # 이미지는 PyTorch 텐서이므로 Numpy 배열로 변환해야 합니다.
  image = image.numpy()

  # PyTorch는 이미지를 (채널, 높이, 너비) 형식으로 저장하지만, 
  # matplotlib은 (높이, 너비, 채널) 형식으로 필요하므로 이미지의 축을 바꿉니다.
  image = np.transpose(image, (1, 2, 0))

  # 이미지를 출력합니다.
  plt.imshow(image.squeeze(), cmap='gray')
  plt.title(f"Label: {label}")
  plt.show()

In [ ]:
label = train_dset[0][1]
print(label)

9


## 2. 심층신경망 구현

In [ ]:
class FashionDNN(nn.Module):
    def __init__(self):
        super(FashionDNN, self).__init__()  # PyTorch의 기본 클래스인 nn.Module의 초기화 메서드를 호출
        self.fc1 = nn.Linear(in_features=784, out_features=256)  # 784개의 입력 특성을 받아 256개의 출력 특성을 생성하는 선형 레이어
        self.drop = nn.Dropout(0.25)  # 학습 중에 노드의 25%를 무작위로 끄는 드롭아웃 레이어
        self.fc2 = nn.Linear(in_features=256, out_features=128)  # 256개의 입력 특성을 받아 128개의 출력 특성을 생성하는 두 번째 선형 레이어
        self.fc3 = nn.Linear(in_features=128, out_features=10)  # 128개의 입력 특성을 받아 10개의 출력 특성을 생성하는 세 번째 선형 레이어

    def forward(self, input_data):
        out = input_data.view(-1, 784)  # 입력 데이터의 형태를 변경, 각 이미지를 1차원 벡터로 변환
        out = F.relu(self.fc1(out))  # 첫 번째 선형 레이어를 통과한 후 ReLU 활성화 함수 적용
        out = self.drop(out)  # 드롭아웃 레이어 적용
        out = F.relu(self.fc2(out))  # 두 번째 선형 레이어를 통과한 후 ReLU 활성화 함수 적용
        out = self.fc3(out)  # 세 번째 선형 레이어를 통과한 결과를 반환 (이 값은 클래스에 대한 로짓을 나타냄)
        return out
